In [17]:
# https://keras.io/
from tensorflow import keras
import labelbox
import json
import os
import urllib
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from urllib.parse import urlparse, unquote
import matplotlib.pyplot as plt
import config
import requests
import pickle

In [18]:
# Enter your Labelbox API key here
LB_API_KEY = config.LB_API_KEY
# Create Labelbox client
lb = labelbox.Client(api_key=LB_API_KEY)
# Get project by ID
project = lb.get_project('clh13dzrw0l4a07zmf36u97rd')
# Export labels created in the selected date range as a json file:
labels = project.export_labels(download = True, start="2023-04-02", end="2023-05-02")

In [19]:
def preprocess_data(data, size=(128, 128), save_to_folder=True):
    images = []
    labels = []
    annotations = []

    if save_to_folder:
        if not os.path.exists('temp_images'):
            os.makedirs('temp_images')
        if not os.path.exists('temp_data'):
            os.makedirs('temp_data')

    for entry in data:
        try:
            img_url = entry['Labeled Data']
            response = requests.get(img_url, stream=True)
            response.raise_for_status()

            img_bytes = response.content
            img_array = np.frombuffer(img_bytes, dtype=np.uint8)
            img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

            if img is None:
                print(f'Nie można wczytać obrazu: {img_url}')
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            bbox = entry['Label']['objects'][0]['bbox']
            if bbox['height'] <= 0 or bbox['width'] <= 0:
                print(f'Nieprawidłowe współrzędne obrazu: {img_url}')
                continue

            plate = img[bbox['top']:bbox['top']+bbox['height'], bbox['left']:bbox['left']+bbox['width']]

            if plate.size == 0:
                print(f'Nie można przeskalować obrazu: {img_url}')
                continue

            # Scaling the image
            scale_ratio_y = size[0] / img.shape[0]
            scale_ratio_x = size[1] / img.shape[1]
            img_resized = cv2.resize(img, size)

            # Scaling the bounding box coordinates
            bbox_scaled = {
                'top': int(bbox['top'] * scale_ratio_y),
                'left': int(bbox['left'] * scale_ratio_x),
                'height': int(bbox['height'] * scale_ratio_y),
                'width': int(bbox['width'] * scale_ratio_x),
            }

            if save_to_folder:
                img_name = entry['External ID']
                img_path = os.path.join('temp_images', img_name)
                cv2.imwrite(img_path, cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR))

            images.append(img_resized)
            labels.append(entry['External ID'])
            annotations.append(bbox_scaled)

        except Exception as e:
            print(f'Wystąpił błąd podczas przetwarzania obrazu: {img_url}')
            print(f'Szczegóły błędu: {str(e)}')

    if save_to_folder:
        with open('temp_data/images.pkl', 'wb') as f:
            pickle.dump(images, f)
        with open('temp_data/annotations.pkl', 'wb') as f:
            pickle.dump(annotations, f)
        with open('temp_data/labels.pkl', 'wb') as f:
            pickle.dump(labels, f)


In [20]:
preprocess_data(labels)
print("Done")

Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG
Invalid SOS parameters for sequential JPEG


Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2Fc749a4e6-7093-d94d-c74c-a846f0ba8bbc-EL_4R071.jpeg?Expires=1684340621301&KeyName=labelbox-assets-key-3&Signature=Je2nzWU_VSOrbyenfgwh59FmmNs
Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F6d137c4d-119e-cafc-5c1d-558f0a345135-EL_9S610.jpeg?Expires=1684340621303&KeyName=labelbox-assets-key-3&Signature=MCeqANydoYTmJ70JJlJKEFdGXis


Invalid SOS parameters for sequential JPEG


Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F38f4439b-7684-bd11-52b3-b972cb230283-ETM_4LR7.jpeg?Expires=1684340621306&KeyName=labelbox-assets-key-3&Signature=VbIUW-wLBq1AeBI-jhbtm6rM_D4
Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F4abd8c72-3c76-5498-92ba-d70bcf7e5549-KMY_04623.jpeg?Expires=1684340621326&KeyName=labelbox-assets-key-3&Signature=NzMvqgZa8tdmVFW7pPAiLEONX74


Invalid SOS parameters for sequential JPEG


...

Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F941fbb1f-c680-175d-c1af-61af362f5cd3-PTU_KC62.jpeg?Expires=1684340622125&KeyName=labelbox-assets-key-3&Signature=a0bxHZqJotIFwPIX4Vs3PH659pM
Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F0fe6c2c1-9ca6-ccc2-13b6-7a2c2857191f-RZ_6299V.jpeg?Expires=1684340622129&KeyName=labelbox-assets-key-3&Signature=AbsYAGiOXWI2kEYQmEt3BpKu63M
Wystąpił błąd podczas przetwarzania obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2Fbeb04431-06fc-364a-a9dd-63fec1a0684e-SGL_UW12.jpeg?Expires=1684340622132&KeyName=labelbox-assets-key-3&Signature=z7_sIdDGQet0OZ8kZc4SEuqn50U
Szczegóły błędu: list index out of range
Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F9e908c53-e92f-d976-e9e8-ee896d6e1654-SJ_3361E.jpeg?Expires=1684340622133&KeyName=labelbox-assets-key-3&Signature=fjks7ITHxP5Oj9j7w-53LTNXVro


Invalid SOS parameters for sequential JPEG


Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2Facf1160a-504a-8960-53a3-671d420e534f-TSK_42484.jpeg?Expires=1684340622140&KeyName=labelbox-assets-key-3&Signature=ZbogwAjSeRRMC5roY6RmFMCYwz0
Nie można przeskalować obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F12cd53e4-1ca5-633e-b50d-18f736460143-WE_4M827.jpeg?Expires=1684340622142&KeyName=labelbox-assets-key-3&Signature=h6UHT39bfaEr5uHZAlUAw9qqZUw
Wystąpił błąd podczas przetwarzania obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2Fbbeda194-dd26-666d-c8fb-a4e8859c0cce-WJ_6122H.jpeg?Expires=1684340622145&KeyName=labelbox-assets-key-3&Signature=EFTfB3w4qlto7RWTdtgEsGcwSQE
Szczegóły błędu: list index out of range


Invalid SOS parameters for sequential JPEG


Wystąpił błąd podczas przetwarzania obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F8bc66ee3-98cf-e325-16fb-b712127f92d3-WW_384TA.jpeg?Expires=1684340622152&KeyName=labelbox-assets-key-3&Signature=TX0zjIOupQxwxTst5eO7QXYFGTg
Szczegóły błędu: list index out of range
Wystąpił błąd podczas przetwarzania obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F9dcd2728-2a5f-a9a1-75d9-405683635d11-WY_7508N.jpeg?Expires=1684340622155&KeyName=labelbox-assets-key-3&Signature=vU5llQ_RrvLUWcK4D8-OEWrlEZE
Szczegóły błędu: list index out of range
Wystąpił błąd podczas przetwarzania obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F36e47294-8457-bad9-cc56-31026dfc5b9a-WZ_520FN.jpeg?Expires=1684340622155&KeyName=labelbox-assets-key-3&Signature=URyQb2xm3UNgjqxKtVkac8O-DF4
Szczegóły błędu: list index out of range
Wystąpił błąd podczas przetwarzania obrazu: https://storage.labelbox.com/clh11uoed0m9x07x44c1a16av%2F2fc73557-691e-6a72-edbf-2e1baae85b96-ZGL_42813.jpe

In [11]:
def load_data():
    if os.path.exists('temp_data/images.pkl') and os.path.exists('temp_data/annotations.pkl') and os.path.exists('temp_data/labels.pkl'):
        with open('temp_data/images.pkl', 'rb') as f:
            images = pickle.load(f)
        with open('temp_data/annotations.pkl', 'rb') as f:
            annotations = pickle.load(f)
        with open('temp_data/labels.pkl', 'rb') as f:
            labels = pickle.load(f)
            
        return np.array(images), np.array(annotations), np.array(labels)

In [21]:
images, annotations, labels = load_data()

In [44]:
def create_annotations_structure(images, annotations, labels):
    annotations_structure = []
    
    for image, annotation, label in zip(images, annotations, labels):
        height, width, _ = image.shape
        objects = []

        obj = {
            'class': 'plate',
            'x_min': annotation['left'],
            'y_min': annotation['top'],
            'x_max': annotation['left'] + annotation['width'],
            'y_max': annotation['top'] + annotation['height']
        }
        objects.append(obj)
        
        ann_structure = {
            'filename': label,
            'width': width,
            'height': height,
            'objects': objects
        }
        annotations_structure.append(ann_structure)
    
    return annotations_structure

In [45]:
annotations_structure = create_annotations_structure(images, annotations, labels)

In [24]:
def convert_to_yolo_format(annotations, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    class_mapping = {'plate': 0}  # mapowanie klasy na indeks (w tym przypadku mamy tylko jedną klasę)

    for annotation in annotations:
        img_path = annotation['filename']
        img_width = annotation['width']
        img_height = annotation['height']
        objects = annotation['objects']

        yolo_labels = []
        for obj in objects:
            class_name = obj['class']
            class_index = class_mapping[class_name]

            x_min, y_min, x_max, y_max = obj['x_min'], obj['y_min'], obj['x_max'], obj['y_max']

            # Obliczanie współrzędnych środkowego punktu, szerokości i wysokości Bounding Box
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            width = x_max - x_min
            height = y_max - y_min

            # Normalizacja wartości do zakresu 0-1
            x_center /= img_width
            y_center /= img_height
            width /= img_width
            height /= img_height

            yolo_label = f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
            yolo_labels.append(yolo_label)

        # Zapis etykiet do pliku .txt
        base_filename = os.path.splitext(os.path.basename(img_path))[0]
        output_path = os.path.join(output_dir, f"{base_filename}.txt")
        with open(output_path, 'w') as f:
            f.write("\n".join(yolo_labels))

In [46]:
convert_to_yolo_format(annotations=annotations_structure, output_dir="yolo_format_plate")

In [49]:
images_folder = 'temp_images'

# Pobierz listę wszystkich plików w folderze
all_files = os.listdir(images_folder)

# Stwórz listę zawierającą tylko ścieżki do plików z rozszerzeniem .jpg
image_paths = [os.path.join(images_folder, file) for file in all_files if file.endswith('.jpeg')]

In [51]:
train_images, val_images = train_test_split(image_paths, test_size=0.2, random_state=42)

In [55]:
with open('train.txt', 'w') as f:
    for img_path in train_images:
        f.write(f'{img_path}\n')

with open('val.txt', 'w') as f:
    for img_path in val_images:
        f.write(f'{img_path}\n')